In [ ]:
import pulp
import json

# ----- Sets -----
routes = ["r1", "r2", "r3"]  # Bus routes
passengers = ["p1", "p2", "p3"]  # Passengers
bus_stops = ["b1", "b2", "b3", "b4"]  # Bus stops
capacity = {"r1": 10, "r2": 15, "r3": 12}  # Bus capacities

# Feasible routes for each passenger
feasible_routes = {
    "p1": ["r1", "r2"],
    "p2": ["r2"],
    "p3": ["r1", "r3"]
}

# Decision Variables
x = pulp.LpVariable.dicts("PassengerRoute", [(i, j) for i in passengers for j in routes], cat="Binary")
S = pulp.LpVariable.dicts("StopAssignment", [(i, j, k) for i in passengers for j in routes for k in bus_stops], cat="Binary")



In [ ]:
# Optimization Problem
prob = pulp.LpProblem("MaximizePassengers", pulp.LpMaximize)

# Objective: Maximize number of assigned passengers
prob += pulp.lpSum(x[i, j] for i in passengers for j in routes)

# Constraints
for i in passengers:
    prob += pulp.lpSum(x[i, j] for j in feasible_routes[i]) <= 1  # Each passenger chooses max 1 route

for j in routes:
    for k in bus_stops:
        prob += pulp.lpSum(x[i, j] * S[i, j, k] for i in passengers) <= capacity[j]  # Capacity constraint

# Solve Optimization
prob.solve()

# Store Results
assignment_results = {"assignments": []}
for i in passengers:
    for j in routes:
        if x[i, j].varValue == 1:  # If passenger assigned to route j
            assigned_stops = [k for k in bus_stops if S[i, j, k].varValue == 1]
            assignment_results["assignments"].append({"passenger": i, "route": j, "stops": assigned_stops})

# Save results to JSON file
with open("optimized_routes.json", "w") as file:
    json.dump(assignment_results, file, indent=4)

print("Optimization complete. Results saved to optimized_routes.json")